In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,True
1,15810944,35,20000,0,True
2,15668575,26,43000,0,False
3,15603246,27,57000,0,False
4,15804002,19,76000,0,True
...,...,...,...,...,...
395,15691863,46,41000,1,False
396,15706071,51,23000,1,True
397,15654296,50,20000,1,False
398,15755018,36,33000,0,True


In [6]:
dataset=dataset.drop("User ID",axis=1)

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[['Age','EstimatedSalary','Gender_Male']]

In [10]:
independent.shape

(400, 3)

In [11]:
dependent=dataset[['Purchased']]

In [12]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini','entropy'],'max_features':['auto','sqrt','log2'],'n_estimators':[10,100]}
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\saran\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
14 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\saran\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\saran\anaconda3\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Users\saran\anaconda3\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\saran\anaconda3\Lib\site-packages\

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [16]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)

In [17]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)

In [18]:
from sklearn.metrics import classification_report
clf=classification_report(y_test,grid_predictions)

In [19]:
print("Report:\n",clf)

Report:
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        85
           1       0.88      0.92      0.90        49

    accuracy                           0.93       134
   macro avg       0.92      0.92      0.92       134
weighted avg       0.93      0.93      0.93       134



In [20]:
print("confusion matrix:\n",cm)

confusion matrix:
 [[79  6]
 [ 4 45]]


In [21]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1 macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1 macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 100}: 0.9256751954513147


In [22]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9668667466986794

In [23]:
table=pd.DataFrame.from_dict(re)

In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000000,0.000000e+00,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.003126,6.251240e-03,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.046877,8.869684e-07,0.025002,0.007655,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.831253,0.831098,0.851527,0.943041,0.860812,0.041932,8
3,0.375019,1.397601e-02,0.050003,0.006250,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.847141,0.870047,0.851527,0.925272,0.943699,0.887537,0.039529,3
4,0.037502,7.654266e-03,0.031252,0.017117,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.847141,0.868752,0.832483,0.905069,0.942166,0.879122,0.039883,6
5,0.371894,2.500105e-02,0.050003,0.006250,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.888107,0.851527,0.925272,0.962264,0.898930,0.040146,2
6,0.000000,0.000000e+00,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.000000,0.000000e+00,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.046877,9.882947e-03,0.018751,0.006250,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.887346,0.808392,0.870362,0.906166,0.903610,0.875175,0.035778,7
9,0.381269,7.654850e-03,0.050003,0.006250,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.867478,0.907177,0.851527,0.925272,0.962264,0.902744,0.039841,1


In [25]:
age_input=int(input("age:"))
Es_salary_input=int(input("Estimated salary:"))
gender_input=int(input("Gender male or female:"))

age:20
Estimated salary:9000
Gender male or female:1


In [26]:
future_predictions=grid.predict([[age_input,Es_salary_input,gender_input]])
print("future predictions:{}".format(future_predictions))

future predictions:[1]
